# Capstone Project: Board Game Rule Helper

**1. Use Case & Problem Statement**

Board games often have complex rulebooks. During a game, players frequently need to clarify specific rules, like scoring conditions, action limitations, or turn sequences. Manually searching through a PDF or physical rulebook can be time-consuming and disrupt the flow of the game.

This project aims to solve this problem by creating a "Board Game Rule Helper". Users can ask questions about the game's rules in natural language (e.g., "How many cards do I draw at the start of my turn?", "What happens if I land on the red space?"), and the system will provide answers directly sourced from the game's rulebook.

**2. Solution using Generative AI**

We will implement a simple Retrieval Augmented Generation (RAG) pipeline using Google's Generative AI models, accessed via the `google-genai` Python SDK (v1.7.0 pattern). We will use an explicit `genai.Client` for interactions. The core idea remains:

1.  **Process the Rulebook:** Load the game's rules text and break it into manageable chunks.
2.  **Embed the Knowledge:** Convert each rule chunk into a numerical vector (embedding) that captures its semantic meaning using `client.models.embed_content`. This uses the **Embeddings** capability.
3.  **Store & Retrieve:** Store these embeddings. When a user asks a question, embed their question and search the stored rule embeddings to find the most relevant chunks based on semantic similarity. This simulates the core concept of a **Vector Search / Vector Store**.
4.  **Generate Grounded Answer:** Provide the user's question along with the *retrieved relevant rule chunks* as context to a powerful generative model (like Gemini 1.5 Flash) using `client.models.generate_content`. This is the **Retrieval Augmented Generation (RAG)** step, ensuring answers are grounded in the source material.

**3. Gen AI Capabilities Demonstrated**

This notebook will explicitly demonstrate the following three capabilities using the `google-genai` client pattern:

1.  **Embeddings:** Using `client.models.embed_content` with the `models/text-embedding-004` model.
2.  **Vector Search / Vector Store (Concept):** Simulating vector search using cosine similarity.
3.  **Retrieval Augmented Generation (RAG):** Using `client.models.generate_content` with retrieved context.

**4. SDK Pattern Update**
This version incorporates the specific setup and usage patterns provided by the user, utilizing `import google.genai as genai`, `genai.Client`, `client.models.*` methods, and includes robust retry logic based on `google.api_core.retry`.

Let's build it!

In [1]:
# Uninstall conflicting packages (as suggested for specific environments like Kaggle)
!pip uninstall -qqy jupyterlab

# Install the specific version of the Google GenAI library
!pip install -U -q "google-genai==1.7.0"

# Standard and Google imports
import google.auth
import json
import os
from datetime import datetime
import google.genai as genai
from google.genai import types # For config objects
from google.api_core import retry # For retry logic

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity # For vector search simulation
import textwrap

# Check the library version
print(f"Using google-genai version: {genai.__version__}")

# --- Authentication and Client Initialization ---
# Using Kaggle secrets - this is specific to the Kaggle environment
# If running elsewhere, adapt key retrieval (e.g., environment variable, google.auth.default())
try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    print("API Key retrieved successfully from Kaggle secrets.")
except ImportError:
    print("Kaggle secrets not available. Trying GOOGLE_API_KEY environment variable.")
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        print("API Key not found in environment variable.")
        # Attempt Application Default Credentials (ADC) as a fallback if appropriate
        try:
            print("Attempting Application Default Credentials (ADC)...")
            credentials, project_id = google.auth.default()
            # Note: genai.Client doesn't directly take credentials object in this version AFAIK
            # ADC usually works implicitly if GOOGLE_API_KEY is *not* set and running in a GCP env.
            # For explicit ADC use, other libraries or configurations might be needed.
            # We'll rely on API key primarily for this Gemini API focused example.
            print("ADC found, but Client requires explicit API Key. Please set GOOGLE_API_KEY.")
            raise ValueError("API Key not found. Please set GOOGLE_API_KEY env var or Kaggle Secret.")
        except google.auth.exceptions.DefaultCredentialsError:
            print("ADC not found.")
            raise ValueError("API Key not found. Please set GOOGLE_API_KEY env var or Kaggle Secret.")
    else:
        print("API Key retrieved successfully from environment variable.")

# Initialize the client with the API Key
# All API calls will now go through this 'client' object
try:
    client = genai.Client(api_key=GOOGLE_API_KEY)
    print("genai.Client initialized successfully.")
except Exception as e:
    print(f"Error initializing genai.Client: {e}")
    raise SystemExit("Client initialization failed.")


# --- Retry Logic (Exactly as Provided) ---
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits (429) or server errors (503).
print("Applying retry logic to client.models.generate_content...")
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

# Check if the method hasn't already been wrapped to avoid wrapping multiple times
if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)
  print("Retry logic applied.")
else:
    print("Retry logic already applied.")

# --- Model Selection ---
# Model names are now passed as parameters to the client methods
TEXT_MODEL_NAME = 'gemini-2.0-flash' # Or another suitable model like 'gemini-2.0-flash' from user example
EMBEDDING_MODEL_NAME = 'models/text-embedding-004' # Keeping the newer embedding model

print(f"Using text model ID: {TEXT_MODEL_NAME} (passed during generation)")
print(f"Using embedding model ID: {EMBEDDING_MODEL_NAME} (passed during embedding)")

# --- Helper function for printing ---
def print_justified(text, width=80):
  """Prints text with justified alignment."""
  print(textwrap.fill(text, width=width))

print("\nSetup complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-lsp 3.10.2 requires jupyterlab<4.0.0a0,>=3.1.0, which is not installed.
Using google-genai version: 1.7.0
API Key retrieved successfully from Kaggle secrets.
genai.Client initialized successfully.
Applying retry logic to client.models.generate_content...
Retry logic applied.
Using text model ID: gemini-2.0-flash (passed during generation)
Using embedding model ID: models/text-embedding-004 (passed during embedding)

Setup complete.


In [2]:
# --- Optional: Cell to Generate Sample Rulebook Text ---

# Make sure the 'client' object and TEXT_MODEL_NAME from Cell 1 are available

print("Generating fictional board game rules...")

# Craft a prompt asking for rules
generation_prompt = """
Generate a set of concise rules for a fictional fantasy board game called "Dragon Keep Treasure Hunt".
The rules should be suitable for a simple RAG demonstration.
Include the following sections clearly marked:

**Objective:** (What is the goal?)
**Game Setup:** (List 5-6 simple setup steps)
**Player Turn:** (Describe 3-4 distinct actions a player can take, e.g., Move, Search, Use Item, Rest. Mention any costs like Action Points)
**Movement:** (Explain how movement works, e.g., dice roll, map spaces)
**Searching:** (Explain how searching for treasure works, e.g., dice roll, special spaces)
**Combat:** (Very simple combat rules, e.g., against Goblins, maybe rolling a die vs a target number)
**Treasure Chests:** (What do players find in them? e.g., Gold, Items, Traps)
**Winning the Game:** (How does a player win?)

Keep the rules clear, relatively simple, and use double newlines between distinct paragraphs or sections for easy parsing later.
"""

try:
    # Use the client to generate the rules
    response = client.models.generate_content(
        model=TEXT_MODEL_NAME, # Use the text generation model
        contents=generation_prompt
        # Add safety settings or generation config if needed
    )

    # Extract the generated text
    generated_rules = response.text
    print("\n--- Generated Rules ---")
    print(generated_rules)
    print("\n--- End Generated Rules ---")
    print("\nCopy the text between '--- Generated Rules ---' and '--- End Generated Rules ---' and paste it into the RULEBOOK_TEXT variable in Cell 2.")

except Exception as e:
    print(f"An error occurred during rule generation: {e}")
    # Provide fallback generic rules if generation fails
    generated_rules = """
**Fallback: Simple Adventure Game Rules**

**Objective:** Be the first player to collect 3 Magic Gems and return to the Starting Village.

**Game Setup:** Place the game board. Shuffle Item cards. Each player takes a Hero pawn and 5 Gold. Place pawns in the Starting Village.

**Player Turn:** You have 3 Action Points (AP) per turn. Spend AP on: Move (1 AP per space), Search (2 AP), Fight (2 AP).

**Movement:** Move your Hero pawn one adjacent space per 1 AP spent.

**Searching:** On a 'Cave' space, spend 2 AP to draw 1 Item card.

**Combat:** If you enter a space with a Monster token, spend 2 AP to fight. Roll one die: 4+ defeats the Monster. If defeated, gain 1 Magic Gem. If you roll 1-3, lose 1 Gold and end your turn.

**Winning:** Enter the Starting Village space with 3 Magic Gems in your possession.
    """
    print("\n--- FALLBACK RULES ---")
    print(generated_rules)
    print("\n--- END FALLBACK RULES ---")
    print("\nUsing fallback rules due to generation error. Copy the fallback text into RULEBOOK_TEXT in Cell 2.")


# You can now copy the 'generated_rules' text (either the generated one or the fallback)
# and paste it into Cell 2, replacing the existing RULEBOOK_TEXT definition.

Generating fictional board game rules...

--- Generated Rules ---
**Objective:**

Be the first player to collect 3 Treasure Chests and return them to the Dragon Keep.

**Game Setup:**

1. Place the game board in the center of the table.
2. Each player chooses a player token and places it on the Dragon Keep space.
3. Shuffle the Treasure Chest cards and place them face down in a pile.
4. Shuffle the Goblin cards and place them face down in a pile.
5. Each player rolls a six-sided die. The player with the highest roll goes first.
6. Each player receives 3 Health Points (HP).

**Player Turn:**

On your turn, you may perform up to two actions. Possible actions include:
*   **Move:** Move your token across the board. (Cost: 1 Action Point)
*   **Search:** Attempt to find treasure at your current location. (Cost: 1 Action Point)
*   **Use Item:** Activate the effect of an item card. (Cost: 1 Action Point)
*   **Rest:** Recover 1 HP. (Cost: 1 Action Point)

**Movement:**

Roll a six-sided die

In [3]:
# --- Step 1: Load and Prepare Rule Document ---
# (No changes needed here, same as before)

RULEBOOK_TEXT = """
**Dragon Keep Treasure Hunt Rules**

**Objective:** Be the first player to collect 3 Dragon Gems scattered within the keep and escape through the main gate.

**Game Setup:**
1. Place the Dragon Keep game board on the table.
2. Shuffle the Treasure Deck and place it face down near the board.
3. Shuffle the Encounter Deck (containing Goblins and Traps) and place it beside the Treasure Deck.
4. Each player chooses a Hero pawn and places it on the 'Entrance Hall' starting space.
5. Each player takes 3 Action Point (AP) tokens.
6. The player wearing the most adventurous socks goes first.

**Player Turn:**
On your turn, you start with 3 Action Points (AP). You can spend AP on the following actions in any order until you run out of AP or choose to end your turn:
A) Move: Spend 1 AP to move your Hero pawn to an adjacent room or corridor space on the board.
B) Search: If you are in a room marked with a 'Search' icon, spend 2 AP to draw one card from the Treasure Deck.
C) Disarm Trap: If you encounter a Trap card from the Encounter deck, you may spend 1 AP to attempt to disarm it. Roll a 6-sided die: on a 4+, the trap is disarmed. On a 1-3, suffer the trap's effect.
D) Rest: Spend 1 AP to do nothing (useful if saving AP is not allowed or if you want to end your turn precisely).

**Movement:**
Movement costs 1 AP per space (room or corridor segment) entered. You cannot move through walls or locked doors unless you have a key item. Some spaces may trigger an Encounter card draw upon entering.

**Searching:**
Searching can only be performed in designated rooms (marked with a 'Search' icon) and costs 2 AP. Successfully searching allows you to draw one card from the Treasure Deck.

**Combat:**
If an Encounter card reveals a Goblin, combat begins. Combat does not cost AP itself, but happens immediately. Roll one 6-sided die. On a result of 5 or 6, you defeat the Goblin. On a result of 1-4, you are pushed back one space and lose 1 AP from your *next* turn (if possible). Defeated Goblins are discarded. Some items may modify combat rolls.

**Treasure Chests:**
Cards drawn from the Treasure Deck represent the contents of chests or findings. These can include:
- Gold Pieces (used for certain item effects or potential scoring variants).
- Dragon Gems (needed to win the game).
- Useful Items (Potions, Rope, Keys, Magic Maps).
- Occasionally, a Trap card might be mixed into the Treasure Deck!

**Winning the Game:**
The first player to collect exactly 3 Dragon Gems and then successfully move their Hero pawn onto the 'Entrance Hall' space (where players started) immediately declares victory and wins the game! You must reach the Entrance Hall; simply collecting the 3rd Gem is not enough.
"""

rule_chunks = [chunk.strip() for chunk in RULEBOOK_TEXT.split('\n\n') if chunk.strip()]
df_rules = pd.DataFrame(rule_chunks, columns=['text_chunk'])
print(f"Rulebook loaded and split into {len(df_rules)} chunks.")
print(df_rules.head())

Rulebook loaded and split into 9 chunks.
                                          text_chunk
0                **Dragon Keep Treasure Hunt Rules**
1  **Objective:** Be the first player to collect ...
2  **Game Setup:**\n1. Place the Dragon Keep game...
3  **Player Turn:**\nOn your turn, you start with...
4  **Movement:**\nMovement costs 1 AP per space (...


In [4]:
# --- Step 2: Generate Embeddings (Capability 1) ---
# Use the client object to generate embeddings.

print(f"\n--- Demonstrating Capability 1: Embeddings ---")
print(f"Generating embeddings for {len(df_rules)} rule chunks using client with model {EMBEDDING_MODEL_NAME}...")

# Updated function to use the client.models.embed_content method
def generate_embeddings_batch_client(client_obj, texts, model_name, task_type="RETRIEVAL_DOCUMENT", batch_size=100):
    """Generates embeddings using the client object in batches (Corrected Structure Handling)."""
    all_embeddings = []
    # Ensure task_type is valid for the model, RETRIEVAL_DOCUMENT is usually safe
    config = types.EmbedContentConfig(task_type=task_type)

    print(f"Processing {len(texts)} texts in batches of {batch_size}...")

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        print(f"  Processing batch starting at index {i}...")
        try:
            # Use client.models.embed_content
            result = client_obj.models.embed_content(
                model=model_name,
                contents=batch, # Pass the list of texts directly
                config=config # Pass the config object
            )

            # --- Debugging ---
            # Keep this commented out unless needed again
            # print(f"  DEBUG: Raw result structure for batch {i}: {result}")
            # --- End Debugging ---

            # --- CORRECTED Extraction Logic based on Debug Output ---
            batch_embeddings = None # Initialize for this batch

            # Check if result has an 'embeddings' attribute which is a list
            if hasattr(result, 'embeddings') and isinstance(result.embeddings, list):
                # Extract the 'values' attribute from each ContentEmbedding object in the list
                try:
                    # List comprehension to get the '.values' (the actual vector) from each item
                    # Added check if item exists and has 'values' before accessing
                    batch_embeddings = [item.values for item in result.embeddings if item and hasattr(item, 'values')]

                    if len(batch_embeddings) == len(result.embeddings): # Check if all items yielded a vector
                         print(f"    Successfully extracted {len(batch_embeddings)} vectors via result.embeddings -> item.values.")
                    elif batch_embeddings: # Check if we got at least some vectors
                         print(f"    Warning: Extracted {len(batch_embeddings)} vectors, but expected {len(result.embeddings)}. Some items might lack 'values' or were None.")
                         # For this batch, we might have partial success. The length check below will handle it.
                    else:
                         print(f"    Warning: Found result.embeddings list, but failed to extract any '.values'.")
                         batch_embeddings = None # Ensure failure

                except AttributeError as attr_err:
                     print(f"    ERROR: Failed accessing '.values' attribute within result.embeddings list item: {attr_err}")
                     batch_embeddings = None # Ensure failure path is taken
                except Exception as e:
                     print(f"    ERROR: Unexpected error during extraction from result.embeddings list: {e}")
                     batch_embeddings = None # Ensure failure path is taken
            else:
                 print(f"  ERROR: Result object does not have '.embeddings' attribute or it's not a list.")
                 # print(f"  DEBUG: Type of result object: {type(result)}") # Optional debug

            # --- Append or Handle Failure ---
            # Check if we got exactly the number of embeddings expected for this batch
            if batch_embeddings is not None and len(batch_embeddings) == len(batch):
                all_embeddings.extend(batch_embeddings)
            else:
                # If extraction failed or length mismatch (e.g. partial success)
                print(f"  ERROR: Failed to extract expected number of embeddings for batch starting at {i}. Expected {len(batch)}, got {len(batch_embeddings) if batch_embeddings else 0}.")
                all_embeddings.extend([None] * len(batch)) # Add None placeholders

        except Exception as e:
            # Catch errors from the API call itself (e.g., network, permissions)
            print(f"  ERROR: Exception during embedding API call for batch starting at {i}: {e}")
            all_embeddings.extend([None] * len(batch)) # Add None placeholders on exception

    print(f"Finished processing all batches. Total embedding results processed: {len(all_embeddings)}")
    return all_embeddings

# Generate embeddings using the client
# Pass the initialized 'client' object
rule_embeddings = generate_embeddings_batch_client(
    client, # Pass the client object
    df_rules['text_chunk'].tolist(),
    EMBEDDING_MODEL_NAME
)

# Add the potentially None-filled embeddings list as a new column
# Using .assign() is generally safer as it returns a new DataFrame copy
try:
    df_rules = df_rules.assign(embedding=rule_embeddings)
except ValueError as e:
    print(f"Error assigning embeddings list (length mismatch?): {e}")
    print(f"Length of DataFrame: {len(df_rules)}, Length of embeddings list: {len(rule_embeddings)}")
    raise SystemExit("Cannot assign embeddings, length mismatch.")


# --- Improved Check and Error Handling ---
print("\n--- Verifying Embeddings Post-Assignment ---")
print(f"DataFrame shape after assigning embeddings column: {df_rules.shape}")

# Explicitly count how many embeddings are NOT None in the column
successful_embeddings_count = df_rules['embedding'].notna().sum()
total_chunks = len(df_rules)
print(f"Total chunks processed: {total_chunks}")
print(f"Number of non-null embeddings generated: {successful_embeddings_count}")

# --- Stop Execution Firmly If No Embeddings Succeeded ---
if successful_embeddings_count == 0:
    # Raise a specific error to halt execution
    raise ValueError("No valid embeddings were generated.") # This should stop the cell

# --- Handle Partial Failures ---
elif successful_embeddings_count < total_chunks:
    failed_count = total_chunks - successful_embeddings_count
    print(f"\nWarning: Embedding generation failed for {failed_count} out of {total_chunks} chunks.")
    # Drop rows with failed embeddings
    print("Dropping rows where embedding generation failed...")
    original_len = len(df_rules)
    df_rules = df_rules.dropna(subset=['embedding'])
    print(f"Dropped {original_len - len(df_rules)} rows. Proceeding with {len(df_rules)} valid chunks.")

    # Double-check if DataFrame became empty after dropping
    if df_rules.empty:
         print("ERROR: DataFrame is empty after dropping rows with failed embeddings.")
         raise ValueError("No valid embeddings remaining after cleanup.")

# --- Proceed only if we have at least one valid embedding ---
print(f"\nProcessing successful. Have {len(df_rules)} chunks with valid embeddings.")

# Now, it should be safe to get the dimension because we know df_rules is not empty
# and the 'embedding' column has at least one valid list.
embedding_dim = len(df_rules['embedding'].iloc[0])
print(f"Embedding dimension for {EMBEDDING_MODEL_NAME}: {embedding_dim}")

print("\nDataFrame with text chunks and embeddings (showing first few rows):")
# Create preview only if embeddings exist and are lists/arrays
df_rules['embedding_preview'] = df_rules['embedding'].apply(
    lambda x: np.round(x[:5], 4) if isinstance(x, (list, np.ndarray)) and len(x) > 0 else "N/A"
)
print(df_rules[['text_chunk', 'embedding_preview']].head())


--- Demonstrating Capability 1: Embeddings ---
Generating embeddings for 9 rule chunks using client with model models/text-embedding-004...
Processing 9 texts in batches of 100...
  Processing batch starting at index 0...
    Successfully extracted 9 vectors via result.embeddings -> item.values.
Finished processing all batches. Total embedding results processed: 9

--- Verifying Embeddings Post-Assignment ---
DataFrame shape after assigning embeddings column: (9, 2)
Total chunks processed: 9
Number of non-null embeddings generated: 9

Processing successful. Have 9 chunks with valid embeddings.
Embedding dimension for models/text-embedding-004: 768

DataFrame with text chunks and embeddings (showing first few rows):
                                          text_chunk  \
0                **Dragon Keep Treasure Hunt Rules**   
1  **Objective:** Be the first player to collect ...   
2  **Game Setup:**\n1. Place the Dragon Keep game...   
3  **Player Turn:**\nOn your turn, you start with.

In [5]:
# --- Step 3: Implement Vector Search (Capability 2) ---
# Use the client object to embed the query.

print(f"\n--- Demonstrating Capability 2: Vector Search / Vector Store (Concept) ---")

# Updated function to use the client for query embedding with CORRECT structure handling
def find_relevant_chunks_client(client_obj, query, df_embeddings, embedding_model, top_n=3):
    """Finds the top_n most relevant text chunks using the client (Corrected Query Embedding)."""
    if df_embeddings.empty:
        print("Error: No document embeddings available to search.")
        return pd.DataFrame()
    # Ensure the 'embedding' column actually exists after potential earlier failures
    if 'embedding' not in df_embeddings.columns:
        print("Error: Document DataFrame is missing the 'embedding' column.")
        return pd.DataFrame()


    print(f"\nSearching for chunks relevant to query: '{query}'")

    # 1. Generate embedding for the user query using the client (CORRECTED EXTRACTION)
    query_embedding = None # Initialize
    query_embedding_np = None # Initialize
    try:
        # Prepare config for query embedding
        query_config = types.EmbedContentConfig(task_type="RETRIEVAL_QUERY") # Use correct task type
        query_embedding_response = client_obj.models.embed_content(
            model=embedding_model, # Use the selected embedding model
            contents=query,        # Pass query string directly
            config=query_config
        )

        # --- CORRECTED Extraction Logic for Single Query ---
        # Expect response.embeddings to be a list with one ContentEmbedding object
        if (hasattr(query_embedding_response, 'embeddings') and
            isinstance(query_embedding_response.embeddings, list) and
            len(query_embedding_response.embeddings) > 0 and
            query_embedding_response.embeddings[0] and # Check if the first item exists
            hasattr(query_embedding_response.embeddings[0], 'values')):

            # Access the 'values' of the *first* ContentEmbedding object in the list
            query_embedding = query_embedding_response.embeddings[0].values
            print(f"    Successfully extracted query embedding vector.")
            # Convert to NumPy array and reshape for cosine_similarity
            query_embedding_np = np.array(query_embedding).reshape(1, -1)

        else:
             # This path should ideally not be reached if API call succeeded
             print(f"    ERROR: Failed to extract query embedding. Response structure unexpected.")
             # print(f"    DEBUG: Query response structure: {query_embedding_response}") # Optional Debug
             # return pd.DataFrame() # Handled by check below

    except Exception as e:
        print(f"    ERROR: Exception during query embedding API call: {e}")
        # return pd.DataFrame() # Handled by check below

    # Check if query embedding was successfully extracted and converted
    if query_embedding_np is None:
         print("    Failed to generate or process the query embedding. Cannot proceed with search.")
         return pd.DataFrame()


    # 2. Calculate Cosine Similarity
    try:
        # Get document embeddings ready for calculation
        rule_embeddings_np = np.array(df_embeddings['embedding'].tolist())

        # Check for shape mismatches (should definitely match now if extraction worked)
        if rule_embeddings_np.shape[1] != query_embedding_np.shape[1]:
             print(f"    ERROR: Embedding dimension mismatch AFTER extraction. Query: {query_embedding_np.shape[1]}, Chunks: {rule_embeddings_np.shape[1]}. Check embedding models consistency.")
             return pd.DataFrame()

        similarities = cosine_similarity(query_embedding_np, rule_embeddings_np)[0]

    except ValueError as ve:
        # Catch errors often related to list-of-lists conversion in np.array() if 'embedding' column has Nones
        print(f"    ERROR: ValueError during similarity calculation. Often due to inconsistent data in 'embedding' column: {ve}")
        return pd.DataFrame()
    except Exception as e:
        print(f"    ERROR: Exception during similarity calculation: {e}")
        return pd.DataFrame()


    # 3. Find Top N matches
    try:
        if len(similarities) != len(df_embeddings):
             print(f"    ERROR: Number of similarities ({len(similarities)}) doesn't match number of documents ({len(df_embeddings)}).")
             return pd.DataFrame()

        # Get indices of top N similarities
        top_n_indices = np.argsort(similarities)[::-1][:top_n]

    except Exception as e:
        print(f"    ERROR: Exception during sorting similarities: {e}")
        return pd.DataFrame()

    # 4. Retrieve the corresponding chunks
    relevant_chunks_df = df_embeddings.iloc[top_n_indices].copy()
    relevant_chunks_df['similarity'] = similarities[top_n_indices]

    print(f"Found {len(relevant_chunks_df)} relevant chunks:")
    print(relevant_chunks_df[['text_chunk', 'similarity']])

    return relevant_chunks_df

# --- Example Search ---
if 'embedding' in df_rules.columns:
    user_query = "How do I win the game?"
    # Use the updated function, passing the client
    relevant_docs = find_relevant_chunks_client(client, user_query, df_rules, EMBEDDING_MODEL_NAME)
else:
    print("Embeddings column not found. Skipping vector search example.")
    relevant_docs = pd.DataFrame()


--- Demonstrating Capability 2: Vector Search / Vector Store (Concept) ---

Searching for chunks relevant to query: 'How do I win the game?'
    Successfully extracted query embedding vector.
Found 3 relevant chunks:
                                          text_chunk  similarity
8  **Winning the Game:**\nThe first player to col...    0.666936
1  **Objective:** Be the first player to collect ...    0.535640
7  **Treasure Chests:**\nCards drawn from the Tre...    0.533111


In [6]:
# --- Step 4: Perform Retrieval Augmented Generation (RAG) (Capability 3) ---
# Use the client object to generate the final answer.

print(f"\n--- Demonstrating Capability 3: Retrieval Augmented Generation (RAG) ---")

# NOTE: We don't need to initialize a separate GenerativeModel instance anymore.
# We use the client directly.

# Updated function to use the client.models.generate_content method
def generate_rag_answer_client(client_obj, query, relevant_docs_df, text_model_id):
    """Generates an answer using the client based on the query and retrieved context."""
    if relevant_docs_df.empty:
        print("No relevant documents found...")
        return "I could not find specific rules related to your question..."

    print(f"\nGenerating RAG answer for query: '{query}' using client with model {text_model_id}...")

    # 1. Prepare context (same as before)
    context = "\n---\n".join(relevant_docs_df['text_chunk'].tolist())

    # 2. Construct RAG Prompt (same structure as before)
    rag_prompt = f"""
You are a helpful Board Game Rule Assistant... answer based *only* on the provided rule excerpts...

**Player's Question:**
{query}

**Relevant Rule Excerpts Provided:**
---
{context}
---

**Answer (Based ONLY on the excerpts):**
"""
    # print(f"Prompt length: {len(rag_prompt)}") # Optional debug

    # 3. Generate the answer using client.models.generate_content
    # The retry logic applied in Cell 1 automatically handles this call.
    try:
        # Call using the client object
        response = client_obj.models.generate_content(
            model=text_model_id, # Pass the model name string
            contents=rag_prompt   # Pass the full prompt string
            # Add generation_config here if needed (e.g., temperature, safety)
            # config = types.GenerateContentConfig(temperature=0.5)
        )

        # Process response (check structure - client response might differ slightly)
        if not response.candidates:
             print("Warning: Response generated, but no candidates found.")
             return "The model did not generate a valid response candidate."
        # Accessing text might be via response.text shortcut if available and unambiguous
        # Or more robustly through candidates and parts
        try:
            final_answer = response.text # Try direct access first
        except ValueError: # Handle cases where .text isn't straightforward (e.g., multiple candidates/parts, non-text parts)
             print("Warning: Direct .text access failed, checking parts.")
             if response.candidates[0].content.parts:
                  final_answer = "".join(part.text for part in response.candidates[0].content.parts if hasattr(part, 'text'))
             else:
                  final_answer = "Model generated response structure without text parts."

        if not final_answer:
             final_answer = "Model returned empty text content."


    except Exception as e:
        print(f"Error during RAG content generation with client: {e}")
        # Check if the error object has response details
        error_response = getattr(e, 'response', None)
        print(f"Error details (if available): {error_response}")
        final_answer = f"Sorry, I encountered an error trying to answer: {e}"

    print("\nFinal Answer (Generated by RAG model):")
    print_justified(final_answer)
    return final_answer

# --- Example RAG Usage ---
if not relevant_docs.empty:
    # Use the updated function, passing the client and the text model ID string
    final_answer = generate_rag_answer_client(client, user_query, relevant_docs, TEXT_MODEL_NAME)
else:
    print(f"\nSkipping RAG generation for '{user_query}' as no relevant documents were found.")

# --- Try another query ---
user_query_2 = "How much fuel does exploring cost?"
if 'embedding' in df_rules.columns:
    relevant_docs_2 = find_relevant_chunks_client(client, user_query_2, df_rules, EMBEDDING_MODEL_NAME, top_n=2)
    if not relevant_docs_2.empty:
        # Use the updated function
        final_answer_2 = generate_rag_answer_client(client, user_query_2, relevant_docs_2, TEXT_MODEL_NAME)
    else:
        print(f"\nSkipping RAG generation for '{user_query_2}' as no relevant documents were found.")
else:
    print("Embeddings column not found. Skipping second vector search example.")


--- Demonstrating Capability 3: Retrieval Augmented Generation (RAG) ---

Generating RAG answer for query: 'How do I win the game?' using client with model gemini-2.0-flash...

Final Answer (Generated by RAG model):
To win, you must be the first player to collect exactly 3 Dragon Gems and then
move your Hero pawn onto the 'Entrance Hall' space. Collecting the 3 Dragon Gems
is not enough; you must also reach the Entrance Hall.

Searching for chunks relevant to query: 'How much fuel does exploring cost?'
    Successfully extracted query embedding vector.
Found 2 relevant chunks:
                                          text_chunk  similarity
5  **Searching:**\nSearching can only be performe...    0.499246
4  **Movement:**\nMovement costs 1 AP per space (...    0.496962

Generating RAG answer for query: 'How much fuel does exploring cost?' using client with model gemini-2.0-flash...

Final Answer (Generated by RAG model):
The provided rules do not mention fuel. Movement costs 1 AP per s

# 4. Conclusion & Limitations

**Conclusion**

This notebook successfully demonstrated a simple Retrieval Augmented Generation (RAG) pipeline to create a "Board Game Rule Helper", utilizing the `google-genai` (v1.7.0 pattern) Python SDK and its client-based interaction model. We applied three key Generative AI capabilities:

1.  **Embeddings:** via `client.models.embed_content` to represent rulebook sections.
2.  **Vector Search (Concept):** via cosine similarity to retrieve relevant sections.
3.  **Retrieval Augmented Generation (RAG):** via `client.models.generate_content` to generate context-grounded answers.

The implementation now accurately reflects the intended client-based interaction pattern, includes robust retry logic, and correctly handles API calls for embedding and generation based on the confirmed response structures. The system effectively processes natural language questions, finds relevant rule snippets, and synthesizes context-aware answers sourced directly from the provided ruleset.

**Limitations & Future Work**

* **Basic Chunking:** Simple paragraph splitting was used. More advanced semantic chunking could improve retrieval relevance for complex rulebooks.
* **Simulated Vector Search:** Calculating cosine similarity across all chunks is inefficient for large documents. A real-world application would integrate an optimized vector database (like FAISS, ChromaDB, Pinecone, or Vertex AI Vector Search) for faster, scalable retrieval.
* **Small Rulebook:** The demo used a relatively short, generated rule set. Performance and accuracy should be tested on larger, official rulebooks.
* **Context Window Limits:** While Gemini 1.5 models have large context windows, extremely long rulebooks or very complex queries requiring many retrieved chunks might still pose challenges, potentially requiring strategies like re-ranking or iterative context feeding.
* **Noisy Retrievals:** Vector search can occasionally return chunks that are semantically similar but not perfectly relevant to the specific nuance of a question. Advanced RAG might involve re-ranking retrieved chunks or more sophisticated prompt engineering.
* **Evaluation:** No formal evaluation (like **Gen AI Evaluation**) was performed to quantitatively measure the accuracy, relevance, and factuality of the generated answers against the source rulebook. This would be a critical step for assessing production readiness.

Despite these limitations, this project serves as a functional proof-of-concept demonstrating how the RAG pattern and associated `google-genai` capabilities can be effectively applied to create useful, grounded information retrieval applications.